In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ai-mathematical-olympiad-progress-prize-2/reference.csv
/kaggle/input/ai-mathematical-olympiad-progress-prize-2/sample_submission.csv
/kaggle/input/ai-mathematical-olympiad-progress-prize-2/AIMO_Progress_Prize_2_Reference_Problems_Solutions.pdf
/kaggle/input/ai-mathematical-olympiad-progress-prize-2/test.csv
/kaggle/input/ai-mathematical-olympiad-progress-prize-2/kaggle_evaluation/aimo_2_gateway.py
/kaggle/input/ai-mathematical-olympiad-progress-prize-2/kaggle_evaluation/__init__.py
/kaggle/input/ai-mathematical-olympiad-progress-prize-2/kaggle_evaluation/aimo_2_inference_server.py
/kaggle/input/ai-mathematical-olympiad-progress-prize-2/kaggle_evaluation/core/templates.py
/kaggle/input/ai-mathematical-olympiad-progress-prize-2/kaggle_evaluation/core/base_gateway.py
/kaggle/input/ai-mathematical-olympiad-progress-prize-2/kaggle_evaluation/core/relay.py
/kaggle/input/ai-mathematical-olympiad-progress-prize-2/kaggle_evaluation/core/kaggle_evaluation.proto
/kaggle/input/ai-ma

In [2]:
pip install print-dict


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 kB 6.4 MB/s eta 0:00:00
  Attempting uninstall: yapf
    Found existing installation: yapf 0.40.2
    Uninstalling yapf-0.40.2:
      Successfully uninstalled yapf-0.40.2
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install peft


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 5.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install simple-dotdict

Note: you may need to restart the kernel to use updated packages.


In [5]:
import torch
import pandas as pd 
import numpy as np 
from print_dict import format_dict
from datasets import load_dataset, Dataset
from transformers import ( AutoModelForCausalLM,AutoTokenizer,
                         Trainer,TrainingArguments,
                         DataCollatorForSeq2Seq
                         )
from peft import  LoraConfig, TaskType,get_peft_model,PeftModel

In [6]:
output_dir='/kaggle/working/'
logging_dir='/kaggle/working/'

model_name='Qwen/Qwen2.5-Math-1.5B-Instruct'
tokenizer_name=model_name


In [7]:
PROMPT_TEMPLATE=("<|im_start|>user\n"
    "PROBLEM:\n{problem}\n"
    "SOLUTION:\n{solution}\n"
    "ANSWER:\nAnswer is {answer}.<|im_end|>\n" 
    "<|im_start|>assistant\n"
    "VERIFICATION:\n{verification}\n"
    "SCORE:\nScore is {score}.<|im_end|>"
)

print('chat template:')
print((0,PROMPT_TEMPLATE)[1])
print(' ')




chat template:
<|im_start|>user
PROBLEM:
{problem}
SOLUTION:
{solution}
ANSWER:
Answer is {answer}.<|im_end|>
<|im_start|>assistant
VERIFICATION:
{verification}
SCORE:
Score is {score}.<|im_end|>
 


In [8]:
#Make dummy deata
class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__



def make_dummy_data():
    num_sample=50
    dummy_df=[]
    
    for i in range(0,num_sample):
        x,y=np.random.randint(0,100,size=2)
        xx,yy=np.random.randint(0,100,size=2)
        v1,v2=np.random.choice(2,size=2,p=[0.4,0.6])
        
        if v1==1:
            s1=f"step.1 Dummy step one: x={x},y={y}"
            w1=f"step.1 Verify: correst"
        else:
            s1=f"step.1 Dummy step one: x={xx},y={yy}"
            w1=f"step.1 Verify: wrong"
            
        if v2==1:
            s2=f"step.2 Dummy step one: x+y={x+y}"
            w2=f"step.2 Verify: correst"
            a=x+y
            
        else:
            s2=f"step.2 Dummy step one: x+y={xx+yy}"
            w2=f"step.2 Verify: wrong"
            a=xx+yy
            
        score=v1*v2 
        example=dict(
        id=f'{i:03d}',
        problem = f'This is a dummy problem. If x = {x} and y = {y}, what is x + y?',
            answer  = f'{a}',
            solution=\
                'This is a dummy solution.\n' + s1 +'\n'  + s2,
            verification=\
                'This is a dummy verification.\n' + w1 +'\n'  + w2,
            score=f'{score}'    
        )
        dummy_df.append(example)
        
    dummy_df=pd.DataFrame(dummy_df)
    return dummy_df 

def data_preprocessing_function(example,tekenizer):
    prompt=("<|im_start|>user\n"
        "PROBLEM:\n{problem}\n"
        "SOLUTION:\n{solution}\n"
        "ANSWER:\nAnswer is {answer}.<|im_end|>\n"
        "<|im_start|>assistant\n" #we let prompt ends with \n
    ).format(problem =example['problem'],
            solution=example['solution'],
            answer=example['answer'],)
    tokenized1=tekenizer(prompt,add_special_tokens=False)
    input_ids1=tokenized1['input_ids']
    
    respond=("VERIFICATION:\n{verification}\n"
        "SCORE:\nScore is {score}.<|im_end|>"
    ).format(verification=example['verification'],
            score=example['score'])
    tokenized2 = tokenizer(respond, add_special_tokens=False)
    input_ids2 = tokenized2['input_ids']
    
    #return tokenized 
    input_ids = input_ids1 + input_ids2
    attention_mask = [1]*len(input_ids)
    labels = [-100]*len(input_ids1) + input_ids2

    return {
        "input_ids": torch.IntTensor(input_ids),
        "attention_mask": torch.IntTensor(attention_mask),
        "labels": torch.IntTensor(labels)
    }

dummy_df=make_dummy_data()
tokenizer = AutoTokenizer.from_pretrained(
    tokenizer_name, trust_remote_code=True
)
dataset = Dataset.from_pandas(dummy_df)
dataset = dataset.map(lambda example: data_preprocessing_function(example, tokenizer),
                      batched=False)
print('one train sample:')
print(format_dict(dataset[0]))
print(' ')



    
        
            
    

tokenizer_config.json:   0%|          | 0.00/7.32k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

one train sample:
{
    'id': '000',
    'problem': 'This is a dummy problem. If x = 83 and y = 70, what is x + y?',
    'answer': '73',
    'solution': 'This is a dummy solution.\nstep.1 Dummy step one: x=83,y=70\nstep.2 Dummy step one: x+y=73',
    'verification': 'This is a dummy verification.\nstep.1 Verify: correst\nstep.2 Verify: wrong',
    'score': '0',
    'input_ids': [
        151644, 872, 198, 9117, 69292, 510, 1986, 374, 264, 17292, 3491, 13, 1416, 856, 284,
        220, 23, 18, 323, 379, 284, 220, 22, 15, 11, 1128, 374, 856, 488, 379, 5267, 50, 45977,
        510, 1986, 374, 264, 17292, 6291, 624, 9520, 13, 16, 50567, 3019, 825, 25, 856, 28, 23,
        18, 7358, 28, 22, 15, 198, 9520, 13, 17, 50567, 3019, 825, 25, 856, 43010, 28, 22, 18,
        198, 11692, 39351, 510, 16141, 374, 220, 22, 18, 13, 151645, 198, 151644, 77091, 198,
        3763, 22872, 510, 1986, 374, 264, 17292, 22901, 624, 9520, 13, 16, 25429, 25, 1829,
        3927, 198, 9520, 13, 17, 25429, 25, 4969, 1

In [9]:
import os 
os.environ['WANDB_DISABLED']='true'

#traing code 
lora_config=LoraConfig(task_type=TaskType.CAUSAL_LM,
                      target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
                      inference_mode=False,
                      r=8,
                      lora_alpha=32,
                      lora_dropout=0.1)

standard_model=AutoModelForCausalLM.from_pretrained(model_name,
                                                   device_map='auto')
model=get_peft_model(standard_model,lora_config)

#set training arguments 
training_args=TrainingArguments( 
    output_dir=output_dir,
    num_train_epochs=10,
    per_device_train_batch_size=4,
    logging_dir=logging_dir,
    logging_steps=10,
    save_steps=100,
    save_total_limit=5,
    fp16=True
)

#create a Trainer instance with the processed data set 
trainer=Trainer( 
    model=model,
    args=training_args,
    train_dataset=dataset,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer,padding=True)
    
)

if 1:
    trainer.train()

config.json:   0%|          | 0.00/656 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Step,Training Loss
10,2.902900
20,1.325700
30,0.445200
40,0.083300
50,0.032700
60,0.036900
70,0.019700
80,0.027100
90,0.020300
100,0.017900


In [10]:
model=PeftModel.from_pretrained( standard_model,'/kaggle/working/checkpoint-130' )
for i in range(10):
    print('-------')
    example=dummy_df.iloc[i]
    
    prompt=( "<|im_start|>user\n"
        "PROBLEM:\n{problem}\n"
        "SOLUTION:\n{solution}\n"
        "ANSWER:\nAnswer is {answer}.<|im_end|>\n"
        "<|im_start|>assistant\n"  # we let prompt ends with \n 
           ).format( problem=example['problem'],
                   solution=example['solution'],
                   answer=example['answer'])
    inputs=tokenizer(prompt,add_special_tokens=False, return_tensors='pt')
    output=model.generate(**inputs, max_length=1024)
    generated_text=tokenizer.decode(output[0],skip_special_tokens=False)
    print(generated_text)
    print(' ')
    
    
    

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1935: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


-------
<|im_start|>user
PROBLEM:
This is a dummy problem. If x = 83 and y = 70, what is x + y?
SOLUTION:
This is a dummy solution.
step.1 Dummy step one: x=83,y=70
step.2 Dummy step one: x+y=73
ANSWER:
Answer is 73.<|im_end|>
<|im_start|>assistant
VERIFICATION:
This is a dummy verification.
step.1 Verify: correst
step.2 Verify: wrong
SCORE:
Score is 0.<|im_end|>
 
-------
<|im_start|>user
PROBLEM:
This is a dummy problem. If x = 21 and y = 35, what is x + y?
SOLUTION:
This is a dummy solution.
step.1 Dummy step one: x=9,y=40
step.2 Dummy step one: x+y=49
ANSWER:
Answer is 49.<|im_end|>
<|im_start|>assistant
VERIFICATION:
This is a dummy verification.
step.1 Verify: wrong
step.2 Verify: wrong
SCORE:
Score is 0.<|im_end|>
 
-------
<|im_start|>user
PROBLEM:
This is a dummy problem. If x = 98 and y = 71, what is x + y?
SOLUTION:
This is a dummy solution.
step.1 Dummy step one: x=98,y=71
step.2 Dummy step one: x+y=169
ANSWER:
Answer is 169.<|im_end|>
<|im_start|>assistant
VERIFICATION:
Th